In [1]:
from _functions import *
#Import libraries and dependencies
investment = 36515165
print(f'Lets get warmed up..')
from dotenv import load_dotenv
from itertools import product
from panel.interact import interact
import datetime
import os
import random
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import panel as pn
pn.extension('plotly')
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from plotly.offline import iplot
from plotly.subplots import make_subplots
from panel import widgets
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
%matplotlib inline
exchange = 'BINANCE'

#Create TradingView variable and activate env variables
print(f'Trading View is my trusted partner, together we will see where your money should go')
from tvDatafeed import TvDatafeed,Interval
load_dotenv()
tradev_id = os.getenv("username")
tradev_secret_key = os.getenv("password")

#Log into TradingView
tv = TvDatafeed(tradev_id, tradev_secret_key, chromedriver_path=None)
print('TradingView Authentication is successfull')

Lets get warmed up..


Trading View is my trusted partner, together we will see where your money should go
TradingView Authentication is successfull


In [2]:
crypataras_picks = ['BTCUSD','ETHUSD','DOGEUSD','LTCUSD','ZRXUSD']

In [3]:
print(f'Extracting most recent prices..')
final_crypto_df = pd.DataFrame()
bars= 1000
for all_picks in crypataras_picks:
    picks_df = tv.get_hist(symbol=all_picks,
                       exchange=exchange,
                       interval=Interval.in_daily,
                       n_bars=bars)
    final_crypto_df = final_crypto_df.append(picks_df)
print(f'Give me a moment while I reformat some datasets to obtain an optimal performance strategy..')

Extracting most recent prices..
Give me a moment while I reformat some datasets to obtain an optimal performance strategy..


In [4]:
#Format dataframe.
#Step 1: extract exchange and ticker from 'symbol' column into separate columns.
final_df_cleaner = final_crypto_df
final_df_cleaner[['exchange', 'ticker']] = final_df_cleaner['symbol'].str.split(':',expand=True)
#Step 2: Extract ticker and close values, reset index
final_df_cleaner = final_df_cleaner[['ticker','close']]
final_df_cleaner = final_df_cleaner[['ticker','close']].reset_index()
#Step3: Pivot data to make ticker values the column headers
final_df_cleaner = final_df_cleaner.pivot(index='datetime',
           columns='ticker',
           values='close')
#Step4: calculate return data
print('Extracted the historical returns for Cryparas picks..')
crypto_pick_returns = final_df_cleaner.pct_change().dropna()
crypto_pick_returns.tail(5)

Extracted the historical returns for Cryparas picks..


ticker,BTCUSD,DOGEUSD,ETHUSD,LTCUSD,ZRXUSD
datetime,,,,,
2022-01-31 18:00:00,0.007233,0.005646,0.037379,0.050175,0.025634
2022-02-01 18:00:00,-0.045768,-0.038596,-0.039129,-0.058338,-0.015385
2022-02-02 18:00:00,0.009821,0.008029,0.006059,0.023871,0.065995
2022-02-03 18:00:00,0.114415,0.069515,0.111386,0.094199,0.059205
2022-02-04 18:00:00,-0.000411,0.000000,0.008996,0.009002,0.012352


In [5]:
#Step5: Calculate Cumulative Returns
print('Getting a feel for each cryptos LTD cumulative returns ..')
cumulative_daily_returns = (1 + crypto_pick_returns).cumprod() -1
cumulative_daily_returns.tail(10)

Getting a feel for each cryptos LTD cumulative returns ..


ticker,BTCUSD,DOGEUSD,ETHUSD,LTCUSD,ZRXUSD
datetime,,,,,
2022-01-26 18:00:00,4.074319,67.942684,17.081002,1.537921,1.912635
2022-01-27 18:00:00,4.155555,67.893996,17.977482,1.595227,1.988107
2022-01-28 18:00:00,4.222854,68.770386,18.389437,1.628783,2.111104
2022-01-29 18:00:00,4.176116,66.774164,18.409479,1.577343,2.034863
2022-01-30 18:00:00,4.249545,67.991372,19.034899,1.599118,2.116872
2022-01-31 18:00:00,4.287513,68.380879,19.783779,1.729529,2.196769
2022-02-01 18:00:00,4.045512,65.703021,18.970524,1.570293,2.147586
2022-02-02 18:00:00,4.095063,66.238592,19.091525,1.631648,2.355311
2022-02-03 18:00:00,4.678014,70.912673,21.329447,1.879546,2.553961


In [6]:
#Step5: Calculate Cumulative Returns
print('Activating Smartfolio program to determine optimal investment strategy..')
print('')
print('Logging returns to normalize dataset..')
log_returns= np.log(abs(cumulative_daily_returns))
log_returns.tail()

Activating Smartfolio program to determine optimal investment strategy..

Logging returns to normalize dataset..


ticker,BTCUSD,DOGEUSD,ETHUSD,LTCUSD,ZRXUSD
datetime,,,,,
2022-01-31 18:00:00,1.455707,4.225093,2.984862,0.547849,0.786988
2022-02-01 18:00:00,1.397608,4.185145,2.942886,0.451262,0.764344
2022-02-02 18:00:00,1.409782,4.193263,2.949245,0.489590,0.856673
2022-02-03 18:00:00,1.542874,4.261449,3.060089,0.631030,0.937645
2022-02-04 18:00:00,1.542375,4.261449,3.069462,0.644727,0.954688


In [7]:
#Calculate volatility and Initialize variables 
print(f'Running multiple scenaries to determine appropriate weights for selected cryptos')

num_of_portfolios= 5000
weight= np.zeros((num_of_portfolios,len(crypataras_picks)))
expected_return=  np.zeros(num_of_portfolios)
expected_volatility= np.zeros(num_of_portfolios)
sharpe_ratio= np.zeros(num_of_portfolios)
expected_return
mean_log_ret= log_returns.mean()
sigma = log_returns.cov()

for i in range (num_of_portfolios):
    # generate random weight vectos 
    w= np.array(np.random.random(len(crypataras_picks)))
        #sum of weights equal to 1 
    w= w/ np.sum(w)
    weight[i,:]= w
    
    # expected log return 
    expected_return[i]= np.sum(mean_log_ret*w)
    
    #expected volatility 
    expected_volatility[i]= np.sqrt(np.dot(w.T,np.dot(sigma,w)))
    
    # sharpe ratio
    sharpe_ratio[i] =  expected_return[i]/expected_volatility[i]
    
print(f'Here are the weights: ')
# Weighted portfolio that sharpe ratio
max_index = sharpe_ratio.argmax()
weight[max_index,:]

Running multiple scenaries to determine appropriate weights for selected cryptos
Here are the weights: 


array([0.01830992, 0.11429924, 0.64498254, 0.01600198, 0.20640632])

In [8]:
# Portfolio weights are exported to a csv that can be used for monteclaro simulation
print(f'Loading Smartfolio weights..')
print(f'')
print(f'')
portfolio_weights = np.array([weight[max_index,:]])
portfolio_weights = portfolio_weights.tolist() 
smartfolio_weights = pd.DataFrame(portfolio_weights)
total_invested=(smartfolio_weights*investment).round(2)
smartfolio_weights.to_csv('csv_data/smartfolio_weights.csv')
smartfolio_weights_df = smartfolio_weights
smartfolio_weights_df.columns = ([crypataras_picks])
print(f' ............and DONE!')

print('')
print('')
print(f'At this moment in time, here is how you should allocate your {investment} investment...') 
print('')
#print(f' The optimal portfolio weight for '
print(f'With a {smartfolio_weights.iloc[0][0]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][0]} should be allocated to {crypataras_picks[0]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][1]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][1]} should be allocated to {crypataras_picks[1]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][2]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][2]} should be allocated to {crypataras_picks[2]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][3]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][3]} should be allocated to {crypataras_picks[3]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][4]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][4]} should be allocated to {crypataras_picks[4]}.')

Loading Smartfolio weights..


 ............and DONE!


At this moment in time, here is how you should allocate your 36515165 investment...

With a 0.018 weight, Cryptara suggests that $668589.72 should be allocated to BTCUSD.

With a 0.114 weight, Cryptara suggests that $4173655.75 should be allocated to ETHUSD.

With a 0.645 weight, Cryptara suggests that $23551643.88 should be allocated to DOGEUSD.

With a 0.016 weight, Cryptara suggests that $584314.98 should be allocated to LTCUSD.

With a 0.206 weight, Cryptara suggests that $7536960.67 should be allocated to ZRXUSD.
